In [2]:
import os
import shutil
import glob
import time
import multiprocessing as mp
from tqdm import tqdm
from google.colab import drive
import sys

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/DL_project')
# --- 설정 ---
# 1. 원본(구글 드라이브)과 대상(코랩 로컬) 경로 설정
GDRIVE_BASE_PATH = "/content/drive/MyDrive/DL_project"
LOCAL_BASE_PATH = "/content/local_ocr_data" # 데이터를 저장할 로컬 폴더 이름

# 복사할 디렉토리 및 파일 목록
# 여기에 필요한 모든 폴더와 파일을 추가하면 됩니다.
items_to_copy = [
    "train_images",
    "train_labels",
    "test_images",
    "test_labels",
    "vocab.json"
]

# 사용할 CPU 코어 수 (전체 코어 사용)
num_processes = mp.cpu_count()

# --- 복사 작업 함수 ---
def copy_file(paths):
    """
    단일 파일을 소스에서 타겟으로 복사하는 함수 (각 프로세스가 실행)
    shutil.copy2는 파일 내용과 메타데이터(수정 시간 등)를 함께 복사합니다.
    """
    source_path, dest_path = paths
    try:
        # 파일이 복사될 디렉토리가 없으면 생성
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copy2(source_path, dest_path)
    except Exception as e:
        # 오류 발생 시 파일 경로와 함께 오류 메시지 출력
        print(f"Error copying {source_path}: {e}")

# --- 메인 실행 로직 ---
if __name__ == '__main__':
    start_time = time.time()

    print(f"복사를 시작합니다. 병렬 처리를 위해 {num_processes}개의 CPU 코어를 사용합니다.")

    # 1. 복사할 모든 파일의 전체 경로 리스트 생성
    file_list = []
    for item in items_to_copy:
        source_item_path = os.path.join(GDRIVE_BASE_PATH, item)
        dest_item_path = os.path.join(LOCAL_BASE_PATH, item)

        if os.path.isdir(source_item_path):
            # source가 디렉토리인 경우, 내부의 모든 파일을 찾습니다.
            # glob.glob(..., recursive=True)는 하위 폴더까지 모두 검색합니다.
            source_files = glob.glob(os.path.join(source_item_path, '**', '*'), recursive=True)
            for src_file in source_files:
                if os.path.isfile(src_file):
                    # 각 파일에 대한 목적지 경로를 계산합니다.
                    relative_path = os.path.relpath(src_file, GDRIVE_BASE_PATH)
                    dest_file = os.path.join(LOCAL_BASE_PATH, relative_path)
                    file_list.append((src_file, dest_file))
        elif os.path.isfile(source_item_path):
            # source가 단일 파일인 경우
            file_list.append((source_item_path, dest_item_path))

    total_files = len(file_list)
    if total_files == 0:
        print("복사할 파일이 없습니다. 경로를 확인해주세요.")
    else:
        print(f"총 {total_files}개의 파일을 복사합니다...")

        # 2. 멀티프로세싱 Pool을 생성하고 작업을 실행
        # Pool: 정해진 수의 프로세스를 만들어 놓고 작업을 분배하는 관리자
        with mp.Pool(processes=num_processes) as pool:
            # tqdm: 작업 진행률을 보여주는 프로그레스 바
            # pool.imap_unordered: 순서에 상관없이 작업이 끝나는 대로 결과를 받아 처리 (효율적)
            list(tqdm(pool.imap_unordered(copy_file, file_list), total=total_files, desc="파일 복사 중"))

        end_time = time.time()
        print(f"\n✅ 모든 데이터 복사 완료! (총 소요 시간: {end_time - start_time:.2f}초)")

Mounted at /content/drive
복사를 시작합니다. 병렬 처리를 위해 8개의 CPU 코어를 사용합니다.
총 71071개의 파일을 복사합니다...


파일 복사 중:  68%|██████▊   | 48183/71071 [2:20:05<1:06:32,  5.73it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/pool.py", line 856, in next
    item = self._items.popleft()
           ^^^^^^^^^^^^^^^^^^^^^
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipython-input-2837675952.py", line 82, in <cell line: 0>
    list(tqdm(pool.imap_unordered(copy_file, file_list), total=total_files, desc="파일 복사 중"))
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
               ^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/pool.py", line 861, in next
    self._cond.wait(timeout)
  File "/usr/lib/python3.12/threading.py", line 355, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", 

TypeError: object of type 'NoneType' has no len()

In [ ]:
# ✨✨✨ 이 부분만 수정 ✨✨✨
import os
import sys
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import pickle
import time
from google.colab import drive

import multiprocessing as mp

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/DL_project')

from dataset.ocr_dataset import OCRDataset
from cu_length_predictor_net_bn import LengthPredictorNetBN
from common.cu_optimizer import Adam

# 데이터 생산자 함수 (기존과 동일)
def data_producer(dataset, queue, batch_size):
    while True:
        x_batch, t_batch = dataset.get_batch(batch_size)
        if x_batch.shape[0] > 0:
            queue.put((x_batch, t_batch))

# 헬퍼 함수 (기존과 동일)
def convert_to_length_labels(text_labels, max_len, pad_id=0):
    length_labels = []
    for label in cp.asnumpy(text_labels):
        true_len_idx = np.where(label == pad_id)[0]
        if len(true_len_idx) > 0:
            length = true_len_idx[0]
            if length == 0: length = 1
            length_labels.append(length - 1)
        else:
            length_labels.append(max_len - 1)
    return cp.array(length_labels, dtype=cp.int32)

# 1. 데이터 로드 및 파라머터 설정
print("데이터 로딩 중...")

# ✨✨✨ [변경점] 기본 경로를 코랩 로컬 경로로 수정 ✨✨✨
BASE_PATH = "/content/local_ocr_data" # 구글 드라이브 대신 로컬 경로 사용
GDRIVE_PROJECT_PATH = "/content/drive/MyDrive/DL_project" # 파라미터 저장용

TRAIN_IMG_PATH = os.path.join(BASE_PATH, "train_images")
TRAIN_LBL_PATH = os.path.join(BASE_PATH, "train_labels")
TEST_IMG_PATH = os.path.join(BASE_PATH, "test_images")
TEST_LBL_PATH = os.path.join(BASE_PATH, "test_labels")
VOCAB_PATH = os.path.join(BASE_PATH, "vocab.json")

IMAGE_WIDTH, IMAGE_HEIGHT = 256, 64
MAX_LABEL_LEN = 25
PAD_ID = 0

# 데이터셋 객체 생성 시 이제 빠른 로컬 경로를 사용합니다.
train_dataset = OCRDataset(TRAIN_IMG_PATH, TRAIN_LBL_PATH, VOCAB_PATH, image_size=(IMAGE_WIDTH, IMAGE_HEIGHT), max_label_len=MAX_LABEL_LEN)
test_dataset = OCRDataset(TEST_IMG_PATH, TEST_LBL_PATH, VOCAB_PATH, image_size=(IMAGE_WIDTH, IMAGE_HEIGHT), max_label_len=MAX_LABEL_LEN)

# (이하 모든 코드는 원래 코드와 완전히 동일합니다)

# 2. 모델, 옵티마이저 생성
network = LengthPredictorNetBN(input_dim=(1, IMAGE_HEIGHT, IMAGE_WIDTH),
                               max_output_len=MAX_LABEL_LEN)
optimizer = Adam(lr=0.0001)

# 3. 학습 하이퍼파라미터 설정
iters_num = 5000
batch_size = 256
train_size = len(train_dataset.image_files)
iter_per_epoch = max(train_size // batch_size, 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 멀티프로세싱 설정
NUM_PRODUCERS = mp.cpu_count()
print(f"사용 가능한 CPU 코어 수: {NUM_PRODUCERS}, {NUM_PRODUCERS}개의 생산자 프로세스를 사용합니다.")
data_queue = mp.Queue(maxsize=NUM_PRODUCERS * 5)
producers = []
for _ in range(NUM_PRODUCERS):
    p = mp.Process(target=data_producer,
                   args=(train_dataset, data_queue, batch_size),
                   daemon=True)
    p.start()
    producers.append(p)

data_time = 0
compute_time = 0

print("\n🚀 글자 수 예측 모델 학습을 시작합니다 (로컬 디스크 I/O + 멀티프로세싱)...")
for i in range(iters_num):
    start_data = time.time()
    x_batch_np, t_text_batch_np = data_queue.get()
    x_batch = cp.asarray(x_batch_np)
    t_text_batch = cp.asarray(t_text_batch_np)
    data_time += time.time() - start_data

    t_len_batch = convert_to_length_labels(t_text_batch, MAX_LABEL_LEN, PAD_ID)

    start_compute = time.time()
    grad = network.gradient(x_batch, t_len_batch)
    optimizer.update(network.params, grad)
    compute_time += time.time() - start_compute

    loss = network.loss(x_batch, t_len_batch)
    train_loss_list.append(cp.asnumpy(loss))

    if (i + 1) % 20 == 0:
        avg_data_time = data_time / 20
        avg_compute_time = compute_time / 20
        print(f"Iter: {i+1} / {iters_num} | Loss: {loss.get():.4f} | "
              f"데이터 로딩+전송: {avg_data_time:.3f}초 | 학습 연산: {avg_compute_time:.3f}초")
        data_time, compute_time = 0, 0

    if (i + 1) % iter_per_epoch == 0:
        epoch_num = (i + 1) // iter_per_epoch

        x_train_sample_np, t_train_text_sample_np = train_dataset.get_batch(100)
        x_train_sample = cp.asarray(x_train_sample_np)
        t_train_text_sample = cp.asarray(t_train_text_sample_np)
        t_train_len_sample = convert_to_length_labels(t_train_text_sample, MAX_LABEL_LEN, PAD_ID)
        train_acc = network.accuracy(x_train_sample, t_train_len_sample)
        train_acc_list.append(cp.asnumpy(train_acc))

        x_test_sample_np, t_test_text_sample_np = test_dataset.get_batch(100)
        x_test_sample = cp.asarray(x_test_sample_np)
        t_test_text_sample = cp.asarray(t_test_text_sample_np)
        t_test_len_sample = convert_to_length_labels(t_test_text_sample, MAX_LABEL_LEN, PAD_ID)
        test_acc = network.accuracy(x_test_sample, t_test_len_sample)
        test_acc_list.append(cp.asnumpy(test_acc))

        print(f"========== EPOCH {int(epoch_num)} ==========")
        print(f"Train Acc: {train_acc.get():.4f} | Test Acc: {test_acc.get():.4f}")
        print("==============================")

for p in producers:
    p.terminate()

# 4. 파라미터 저장 (저장은 다시 구글 드라이브에 합니다)
params_file = os.path.join(GDRIVE_PROJECT_PATH, "length_predictor_params_cupy_local.pkl")
params_cpu = {}
for key, val in network.params.items():
    params_cpu[key] = cp.asnumpy(val)
with open(params_file, 'wb') as f:
    pickle.dump(params_cpu, f)
print(f"\n✅ 학습된 파라미터를 '{params_file}'에 저장했습니다.")

# 5. 그래프 그리기 (기존과 동일)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_list, label='train loss')
plt.xlabel("iterations")
plt.ylabel("loss")
plt.title("Length Predictor: Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_acc_list, label='train acc')
plt.plot(test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Length Predictor: Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

Mounted at /content/drive
데이터 로딩 중...
🚀 OCR 데이터셋 초기화를 시작합니다...


ValueError: image_paths_or_dir는 디렉토리 경로(str) 또는 파일 경로 리스트(list)여야 합니다.